<img src="https://images.unsplash.com/photo-1630342654882-aed9e2e86f24?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=774&q=80" width="1000px" >

# RETAIL ANALYIS 

1. ## Data Cleaning
2. ## RFM Analysis
3. ## Market Segmentation
4. ## Market Basket Analysis
4. ## Conclusion

In [ ]:
#Imports
#Data cleaning 
import pandas as pd
from ast import literal_eval
from datetime import datetime as dt
from numpy import where

In [ ]:
#READ DATA
#users= read_csv('../input/awssampleretaildemostore/users.csv')
users = pd.read_csv('../input/awssampleretaildemostore/raw_users.csv',
                    usecols =[0,1,2,3,4,5,6,7])
#items = read_csv('../input/awssampleretaildemostore/items.csv')
products = pd.read_csv('../input/awssampleretaildemostore/raw_products.csv', 
                       usecols = [0,3,4,5,7,10])
df_lst = pd.read_csv('../input/awssampleretaildemostore/interactions.csv',
                        iterator=True, chunksize=20000)

df = pd.concat(df_lst)    

## Data Cleaning and Exploration

In [ ]:

users.columns = [c.lower() for c in users.columns]

# Parse address information
add_lst = ['address1', 'country', 'city', 'state']
users.addresses = users.addresses.apply(lambda x: [(literal_eval(x[1:-1])).\
                                                   get(i) for i in add_lst])
users[add_lst] = pd.DataFrame(users.addresses.tolist())
users.drop('addresses', axis=1, inplace = True)

In [ ]:
# Clean Interaction data

#Convert timestamp
df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['timestamp'],  unit='s')
df.drop('timestamp', axis = 1, inplace = True)

#Check for duplicates
df.drop_duplicates(inplace = True)


In [ ]:
products.rename(columns={'id': 'item_id', 'name': 'product_name'}, 
               inplace = True)

#Merge all datasets
df = df.merge(products, how='left').merge(
                                    users, how='left', left_on = 'user_id',
                                    right_on = 'id')
df.drop('id', axis = 1, inplace = True) 

#Remove price from non purchasing event_types
browse = ['ProductViewed', 'ProductAdded', 'CartViewed', 'CheckoutStarted']
df['price'] = where(df['event_type'].isin(browse), "", df['price'])
df['price'] = pd.to_numeric(df['price'])


### Missing data 

In [ ]:
df.isnull().sum()
#No missing values

## Products Analysis

In [ ]:
#How many Categories 
df.groupby('category').agg(no_of_customers = ('user_id','nunique'),
                           no_of_products = ('product_name', 'nunique'),
                           revenue_by_category=('price', 'sum'),
                           no_orders = ('price', 'count')
                          ).reset_index()
                        

## Time

In [ ]:
df['date'].dt.month.unique()
#This is 3 Months of data june/july/ August

df['month'] = df['date'].dt.month_name()
              

In [ ]:
df_t = df.groupby('month').agg(no_of_customers = ('user_id','nunique'),
                           no_of_products = ('product_name', 'nunique'),
                           revenue_by_category=('price', 'sum'),
                           no_orders = ('price', 'count')
                          ).reset_index()
#Reorder by Month
sort_order= ['June', 'July', 'August'] #category order
df_t.index = pd.CategoricalIndex(df_t['month'], categories=sort_order, ordered=True)
df_t = df_t.sort_index().reset_index(drop = True)   
df_t                        

In [ ]:
#Weekly
df['day_of_year'] = df['date'].dt.strftime('%j')
df['day_of_week'] = df['date'].dt.strftime('%a')
df